## Imports

In [1]:
import trimesh
import open3d as o3d
import numpy as np
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from open3d.visualization import gui
from open3d.visualization import rendering

C:\Users\Arif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Load and compute the mean curvature

In [2]:
# Load the tooth STL model using Trimesh
mesh_trimesh = trimesh.load_mesh("stlFiles/sinifbirNumaraAlti.stl")

# Get vertices, faces, and normals
vertices = np.array(mesh_trimesh.vertices)
faces = np.array(mesh_trimesh.faces)
normals = np.array(mesh_trimesh.vertex_normals)

dent_mesh = o3d.geometry.TriangleMesh()
dent_mesh.vertices = o3d.utility.Vector3dVector(vertices)
dent_mesh.triangles = o3d.utility.Vector3iVector(faces)
dent_mesh.compute_vertex_normals()
# Compute Mean Curvature using Trimesh
mean_curvature = trimesh.curvature.discrete_mean_curvature_measure(mesh_trimesh, mesh_trimesh.vertices, radius=2)


### The function that extracts the cavity from the tooth model

In [ ]:
def extract_largest_cavity(vertices, faces, cavity_indices):
    # Get unique cavity indices
    unique_cavity_indices = np.unique(cavity_indices)
    
    # Find faces that have all vertices in cavity_indices
    cavity_face_mask = np.isin(faces.ravel(), unique_cavity_indices).reshape(faces.shape)
    cavity_face_indices = np.where(np.all(cavity_face_mask, axis=1))[0]
    cavity_faces = faces[cavity_face_indices]
    
    # Create adjacency matrix for connected component analysis
    edges = set()
    for face in cavity_faces:
        edges.add((face[0], face[1]))
        edges.add((face[1], face[2]))
        edges.add((face[2], face[0]))
    
    # Create sparse adjacency matrix
    row, col = zip(*edges)
    row = np.array(row)
    col = np.array(col)
    data = np.ones_like(row)
    
    # Create sparse matrix with size equal to total vertices 
    # (will be pruned to cavity vertices later)
    adj_matrix = csr_matrix((data, (row, col)), shape=(vertices.shape[0], vertices.shape[0]))
    
    # Find connected components
    n_components, labels = connected_components(csgraph=adj_matrix, directed=False)
    
    # Count vertices in each component
    component_sizes = np.zeros(n_components, dtype=int)
    for label in labels[unique_cavity_indices]:
        component_sizes[label] += 1
    
    # Find the largest component
    largest_component = np.argmax(component_sizes)
    
    # Get vertices from the largest component
    largest_cavity_indices = np.where(labels == largest_component)[0]
    largest_cavity_indices = np.intersect1d(largest_cavity_indices, unique_cavity_indices)
    
    # Create index mapping for new mesh
    index_map = np.zeros(len(vertices), dtype=int)
    for i, idx in enumerate(largest_cavity_indices):
        index_map[idx] = i
    
    # Get faces for largest component
    largest_face_mask = np.isin(cavity_faces.ravel(), largest_cavity_indices).reshape(cavity_faces.shape)
    largest_face_indices = np.where(np.all(largest_face_mask, axis=1))[0]
    largest_cavity_faces = cavity_faces[largest_face_indices]
    
    # Remap face indices
    remapped_faces = np.zeros_like(largest_cavity_faces)
    for i in range(largest_cavity_faces.shape[0]):
        for j in range(3):
            remapped_faces[i, j] = index_map[largest_cavity_faces[i, j]]
    
    # Create and return the largest cavity mesh
    largest_cavity_mesh = o3d.geometry.TriangleMesh()
    largest_cavity_mesh.vertices = o3d.utility.Vector3dVector(vertices[largest_cavity_indices])
    largest_cavity_mesh.triangles = o3d.utility.Vector3iVector(remapped_faces)
    largest_cavity_mesh.compute_vertex_normals()
    
    # Set color for visualization
    cavity_colors = np.ones((len(largest_cavity_indices), 3)) * [0, 1, 0]  # Green
    largest_cavity_mesh.vertex_colors = o3d.utility.Vector3dVector(cavity_colors)
    
    return largest_cavity_mesh, largest_cavity_indices

### The function that extracts the cavity bottom 

In [4]:
def extract_cavity_bottom(largest_cavity_mesh, threshold_percentage=0.1):
    # Get vertices and triangles from the mesh
    cavity_vertices = np.asarray(largest_cavity_mesh.vertices)
    cavity_triangles = np.asarray(largest_cavity_mesh.triangles)
    
    # Calculate z-range
    min_z = np.min(cavity_vertices[:, 2])
    max_z = np.max(cavity_vertices[:, 2])
    z_range = max_z - min_z
    
    # Define a threshold for what constitutes the "bottom"
    # Here we're considering the bottom 10% of the cavity's depth
    z_threshold = min_z + z_range * threshold_percentage
    
    # Find vertices that are in the bottom region
    bottom_vertex_mask = cavity_vertices[:, 2] <= z_threshold
    bottom_vertex_indices = np.where(bottom_vertex_mask)[0]
    
    # Find triangles where all three vertices are in the bottom region
    bottom_triangles_mask = np.isin(cavity_triangles.ravel(), bottom_vertex_indices).reshape(cavity_triangles.shape)
    bottom_triangle_indices = np.where(np.all(bottom_triangles_mask, axis=1))[0]
    
    if len(bottom_triangle_indices) == 0:
        print("No triangles found in the bottom region. Try adjusting the threshold.")
        return None
    
    # Create a new mesh for the bottom surface
    bottom_triangles = cavity_triangles[bottom_triangle_indices]
    
    # Get unique vertices used in the bottom triangles
    unique_vertices = np.unique(bottom_triangles.ravel())
    
    # Create index mapping
    index_map = np.zeros(len(cavity_vertices), dtype=int)
    for i, idx in enumerate(unique_vertices):
        index_map[idx] = i
    
    # Remap triangle indices
    remapped_triangles = np.zeros_like(bottom_triangles)
    for i in range(bottom_triangles.shape[0]):
        for j in range(3):
            remapped_triangles[i, j] = index_map[bottom_triangles[i, j]]
    
    # Create bottom surface mesh
    bottom_mesh = o3d.geometry.TriangleMesh()
    bottom_mesh.vertices = o3d.utility.Vector3dVector(cavity_vertices[unique_vertices])
    bottom_mesh.triangles = o3d.utility.Vector3iVector(remapped_triangles)
    bottom_mesh.compute_vertex_normals()
    
    # Set color for visualization
    bottom_colors = np.ones((len(unique_vertices), 3)) * [0, 0, 1]  # Blue for bottom surface
    bottom_mesh.vertex_colors = o3d.utility.Vector3dVector(bottom_colors)
    
    return bottom_mesh

### The function that calulates the cavity_bottom roughness

In [5]:
def calculate_roughness(mesh):
    mesh_vertices = np.asarray(mesh.vertices)
    mesh_faces = np.asarray(mesh.triangles)
    tri_mesh = trimesh.Trimesh(vertices=mesh_vertices, faces=mesh_faces, process=False)
    normals = tri_mesh.face_normals
    adj = tri_mesh.face_adjacency

    # Compute angle between adjacent face normals
    dot = np.einsum('ij,ij->i', normals[adj[:, 0]], normals[adj[:, 1]])
    angles = np.arccos(np.clip(dot, -1.0, 1.0)) # Mean angle
    angles_deg = np.degrees(angles) # roughness
    return angles_deg


### kavitenin seçilmesi 

In [6]:
cavity_indices = np.where(mean_curvature < 0.4)[0]  # Select all vertices with negative curvature
largest_cavity_mesh, largest_cavity_indices = extract_largest_cavity(vertices, faces, cavity_indices)
cavity_vertices= np.asarray(largest_cavity_mesh.vertices)


### Kavitenin alt kısmının seçilmesi ve kavite yüksekliğinin hesaplanması

In [7]:
cavity_bottom = extract_cavity_bottom(largest_cavity_mesh, threshold_percentage=0.4)

# kavite altının z eksenindeki ortalamasını al
bottom_vertices = np.asarray(cavity_bottom.vertices)
bottom_z_values = bottom_vertices[:, 2]
min_z_mean = np.mean(bottom_z_values) 

# kavite alanının en üstü 
max_z = np.max(cavity_vertices[:, 2])
cavity_depth = max_z - min_z_mean  # Derinlik (Z eksenindeki fark)



### Roughness Hesaplanması

In [8]:

outline_indices = np.where((mean_curvature > 3.0))[0]
roughness = calculate_roughness(cavity_bottom)

In [28]:

def create_cylinder_between_points(point1, point2, radius=0.01, resolution=20, color=None):
    """
    Create a cylinder mesh between two points.
    
    Args:
        point1: Starting point as [x, y, z]
        point2: Ending point as [x, y, z]
        radius: Radius of the cylinder
        resolution: Number of segments for the cylinder
        color: RGB color as [r, g, b] where each value is between 0 and 1
    
    Returns:
        cylinder_mesh: An Open3D mesh representing the cylinder
    """
    # Convert points to numpy arrays
    point1 = np.asarray(point1)
    point2 = np.asarray(point2)
    
    # Calculate the direction vector from point1 to point2
    direction = point2 - point1
    length = np.linalg.norm(direction)
    
    # Create a cylinder along the Z-axis
    cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=length, resolution=resolution)
    
    # Compute the rotation to align with the direction vector
    # First, we need to find the rotation axis and angle
    z_axis = np.array([0, 0, 1])
    direction_normalized = direction / length
    
    # Compute the rotation axis via cross product
    rotation_axis = np.cross(z_axis, direction_normalized)
    
    # If points are aligned along Z-axis, rotation axis will be zero
    if np.linalg.norm(rotation_axis) < 1e-6:
        # Check if direction is parallel or anti-parallel to z_axis
        if direction_normalized[2] > 0:
            # Parallel - no rotation needed
            rotation_matrix = np.eye(3)
        else:
            # Anti-parallel - rotate 180 degrees around X-axis
            rotation_matrix = np.array([
                [1, 0, 0],
                [0, -1, 0],
                [0, 0, -1]
            ])
    else:
        # Normalize rotation axis
        rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
        
        # Compute rotation angle using dot product
        cos_angle = np.dot(z_axis, direction_normalized)
        angle = np.arccos(cos_angle)
        
        # Convert axis-angle to rotation matrix using Rodrigues' formula
        cross_matrix = np.array([
            [0, -rotation_axis[2], rotation_axis[1]],
            [rotation_axis[2], 0, -rotation_axis[0]],
            [-rotation_axis[1], rotation_axis[0], 0]
        ])
        rotation_matrix = np.eye(3) + np.sin(angle) * cross_matrix + (1 - np.cos(angle)) * (cross_matrix @ cross_matrix)
    
    # Rotate the cylinder to align with the direction
    cylinder.rotate(rotation_matrix, center=np.array([0, 0, 0]))
    
    # Translate the cylinder to start at point1
    cylinder.translate(point1 + direction_normalized * (length / 2))
    
    # Set the color if provided
    if color is not None:
        cylinder.paint_uniform_color(color)
    
    return cylinder


### Görüntüleme

In [35]:
# **Çizgiyi tanımlama**
cavity_centroid = np.mean(cavity_vertices, axis=0)
min_z_point = [cavity_centroid[0], cavity_centroid[1], min_z_mean]
max_z_point = [cavity_centroid[0], cavity_centroid[1], max_z]
cylinder_mesh = create_cylinder_between_points(min_z_point, max_z_point)

print("cavity_depth : ",cavity_depth)
print("roughness : ",roughness)
combined = cavity_bottom + cylinder_mesh


# =line_set_to_cylinder_mesh(line_set)
cylinder_mesh.compute_vertex_normals()
o3d.io.write_triangle_mesh("output/cylinder_mesh.stl", cylinder_mesh)


cavity_depth :  2.0301277332356844
roughness :  [18.09196629  3.21604171  7.5380052  ...  9.02044369  0.92320791
 10.34273216]


True